In [1]:
import pandas as pd

## Indice Precios

In [2]:
ix = pd.read_csv('./../data/info/indice_precios_M.csv', index_col = 0)
ix.index = pd.to_datetime(ix.index)
ix_2010 = ix.loc['2010-11-15'].values[0]
ix_2016 = ix.loc['2016-01-15'].values[0]
ipc_10_16 = (ix_2016/ix_2010)

### Cargar info pobreza

In [3]:
frac = 0.01
df_list = []

for Q in ['2010-02-15', '2010-05-15', '2010-08-15', '2010-11-15']:
    file_q = './../data/Pobreza/pobreza_'+str(frac)+'_q'+Q+'.csv'
    dfq = pd.read_csv(file_q, index_col = 0)
    df_list += [dfq]
    
df = pd.concat(df_list)


### Labels aglomerado y Cond. Act y Ocup

In [4]:
aglo_labels = pd.read_csv('./../data/info/aglo_labels.csv')[['AGLOMERADO', 'NOMAGLO']]
condact_labels = pd.read_csv('./../data/info/condact_labels.csv')
cat_ocup_inac_labels = pd.read_csv('./../data/info/cat_ocup_inac_labels.csv')


### Tabla 1: Actividad/Ocup/Inac por aglomerado

In [5]:
info = df.groupby(['AGLOMERADO', 'CONDACT', 'CAT_OCUP', 'CAT_INAC']).size().reset_index()
info = info.merge(aglo_labels).merge(condact_labels).merge(cat_ocup_inac_labels, how = 'left')

table = info.set_index(['NOMAGLO', 'Actividad', 'Categoria'])[0].unstack([-2, -1])
table.sum().loc[table.sum() > 5000]

show = (table[table.sum().loc[table.sum() > 5000].index]/frac/4).fillna(0).astype(int)
# show.to_csv('./../data/results/pob_actividad_aglo.csv')
show.to_csv('./../data/results/Tabla1.csv')

### Tabla 2: Poblacion, ingresos por aglomerado.

In [6]:
info = df.groupby(['AGLOMERADO'])['P47T_persona'].describe().reset_index()
info = info.merge(aglo_labels)#.merge(condact_labels).merge(cat_ocup_inac_labels, how = 'left')
info = info.set_index('NOMAGLO')[['count', 'mean', '25%', '50%', '75%']]

col_mon = ['mean', '25%', '50%', '75%']
info[col_mon] = info[col_mon]/ipc_10_16
info['count'] = info['count']/frac/4
info = info.round(-1).astype(int)
info.columns = ['Poblacion', 'P47T promedio', '25%', '50%', '75%']
show = info.sort_index()
# show.to_csv('./../data/results/P47T_aglo.csv')
show.to_csv('./../data/results/Tabla2.csv')


### Table 3: CBA por aglomerado

In [7]:
# info = df.groupby(['AGLOMERADO'])['CBA'].describe().reset_index()
# info = info.merge(aglo_labels)#.merge(condact_labels).merge(cat_ocup_inac_labels, how = 'left')
# info = info.set_index('NOMAGLO')[['count', 'mean', '25%', '50%', '75%']]

# col_mon = ['mean', '25%', '50%', '75%']
# info[col_mon] = info[col_mon]/ipc_10_16
# info['count'] = info['count']/frac/4
# info = info.round(-1).astype(int)
# info.columns = ['Poblacion', 'CBA promedio', '25%', '50%', '75%']
# info.sort_index()

In [8]:
# df['CBT_'] = df['CBT']/df['CB_EQUIV']

# info = df.groupby(['AGLOMERADO'])['CBT_'].describe().reset_index()
# info = info.merge(aglo_labels)#.merge(condact_labels).merge(cat_ocup_inac_labels, how = 'left')
# info = info.set_index('NOMAGLO')[['count', 'mean', '25%', '50%', '75%']]

# col_mon = ['mean', '25%', '50%', '75%']
# info[col_mon] = info[col_mon]/ipc_10_16
# info['count'] = info['count']/frac/4
# info = info.round(-1).astype(int)
# info.columns = ['Poblacion', 'CBT promedio', '25%', '50%', '75%']
# info.sort_index()

## Columnas de ingresos

In [10]:
import numpy as np

In [11]:
## Labels
col_mon_labels = pd.read_csv('./../data/info/col_mon_labels.csv')
ingresos_labels = pd.read_csv('./../data/info/ingresos_labels.csv')

# Columnas pesos
col_mon = ['P21', 'P47T', 'PP08D1', 'TOT_P12', 'T_VI', 'V12_M', 'V2_M', 'V3_M', 'V5_M']

frac = 0.01
df_list = []

for Q in ['2010-02-15', '2010-05-15', '2010-08-15', '2010-11-15']:
    file_q = './../data/yr_samples/RFReg_'+str(frac)+'_'+Q+'_ARG.csv'
    dfq = pd.read_csv(file_q,
                      usecols = ['AGLOMERADO', 'INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS'] + col_mon)
    df_list += [dfq]
    
df = pd.concat(df_list)

# Pasar log a nominal
df[col_mon] = np.power(10, df[col_mon]) - 1

In [ ]:
# pd.read_csv(file_q, index_col = 0, nrows = 5)


### Tabla 4: Tot Nacional

In [15]:
show = df[col_mon].describe().T

show[['mean', '25%', '50%', '75%']] = (show[['mean', '25%', '50%', '75%']]/ipc_10_16).round()
show['count'] = show['count']/frac/4

show.round().astype(int)#.to_csv('./../data/results/Tabla4.csv')

,count,mean,std,min,25%,50%,75%,max
P21,40005400,1041,6227,0,0,0,1763,215826
P47T,40005400,1384,7517,0,0,665,2116,384170
PP08D1,40005400,971,6034,0,0,0,1646,172661
TOT_P12,40005400,76,1578,0,0,0,0,174124
T_VI,40005400,202,3178,0,0,0,0,345321
V12_M,40005400,13,521,0,0,0,0,43062
V2_M,40005400,158,2255,0,0,0,0,107654
V3_M,40005400,5,1089,0,0,0,0,345321
V5_M,40005400,5,190,0,0,0,0,17547


### Tabla 5: Stats por tipo de ingresos

In [16]:
info = df.merge(ingresos_labels).groupby(['Ingresos'])[col_mon].describe()

show = info.stack(-2).reset_index().merge(col_mon_labels, left_on = 'level_1', right_on = 'variable')
show = show.set_index(['Ingresos', 'Ingresos por:'])[['count', 'mean', '25%', '50%', '75%']].astype(int).sort_index()

show[['mean', '25%', '50%', '75%']] = (show[['mean', '25%', '50%', '75%']]/ipc_10_16).round()
show['count'] = show['count']/frac/4
show.astype(int)#.to_csv('./../data/results/Tabla5.csv')

count  mean   25%   50%   75%
Ingresos      Ingresos por:                                         
Jubilacion    Ayuda Social           4695200     0     0     0     0
              Cuota Alim.            4695200    33     0     0     0
              ITI (lab. + no lab.)   4695200  1770   869  1176  1954
              Indemnizacion          4695200     1     0     0     0
              Jubilacion             4695200  1347   764   977  1466
              No lab.                4695200  1532   798  1029  1710
              Ocup. Ppal.            4695200   215     0     0     0
              Otras ocup.            4695200    14     0     0     0
              Salario                4695200   121     0     0     0
Laborables    Ayuda Social          17717950     0     0     0     0
              Cuota Alim.           17717950     0     0     0     0
              ITI (lab. + no lab.)  17717950  2569  1173  2116  3279
              Indemnizacion         17717950     0     0     0     0
              Jubilacion            17717950     0     0     0     0
              No lab.               17717950     0     0     0     0
              Ocup. Ppal.           17717950  2265  1022  1954  2931
              Otras ocup.           17717950   161     0     0     0
              Salario               17717950  2136   874  1858  2931
Otros No Lab. Ayuda Social            582175     0     0     0     0
              Cuota Alim.             582175   588     0   352   823
              ITI (lab. + no lab.)    582175  2111   547  1115  2345
              Indemnizacion           582175   303     0     0     0
              Jubilacion              582175     0     0     0     0
              No lab.                 582175  1182   337   614  1125
              Ocup. Ppal.             582175   742     0     0   765
              Otras ocup.             582175   142     0     0     0
              Salario                 582175   606     0     0   472
Sin Ingresos  Ayuda Social          16473925     0     0     0     0
              Cuota Alim.           16473925     0     0     0     0
              ITI (lab. + no lab.)  16473925     0     0     0     0
              Indemnizacion         16473925     0     0     0     0
              Jubilacion            16473925     0     0     0     0
              No lab.               16473925     0     0     0     0
              Ocup. Ppal.           16473925     0     0     0     0
              Otras ocup.           16473925     0     0     0     0
              Salario               16473925     0     0     0     0
Subsidio      Ayuda Social            533825   368   164   315   505
              Cuota Alim.             533825    18     0     0     0
              ITI (lab. + no lab.)    533825   551   219   441   704
              Indemnizacion           533825     3     0     0     0
              Jubilacion              533825     0     0     0     0
              No lab.                 533825   392   164   329   527
              Ocup. Ppal.             533825   132     0     0     0
              Otras ocup.             533825    25     0     0     0
              Salario                 533825   122     0     0     0

### Tabla 6: Cant por tipos de ingreso por aglomerados

In [17]:
info = df.groupby(['AGLOMERADO', 'INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS']).size()
info.name = 'Cant.'
info = info.reset_index()
# info = info.rename(columns = {0: 'Cant.'})

info = info.merge(aglo_labels).merge(ingresos_labels)
info = info.set_index(['NOMAGLO', 'Ingresos'])['Cant.'].unstack()#[['count', 'mean', '25%', '50%', '75%']]
info = (info/frac/4).fillna(0).astype(int)

info.sort_index()#.to_csv('./../data/results/Tabla6.csv')

Ingresos,Jubilacion,Laborables,Otros No Lab.,Sin Ingresos,Subsidio
NOMAGLO,,,,,
Bahia Blanca,39925,135750,6675,110325,1025
CABA,460900,1548700,47525,807600,3175
Catamarca,14800,81500,2700,74250,2250
Cdro Rivadavia,13400,75375,700,69725,800
Concordia,16300,62200,2025,66975,3000
Cordoba,170375,679950,36025,550750,10200
Corrientes,28500,149325,7700,149975,4500
Formosa,17525,91225,4375,103475,3700
Fuera de aglom.,1782250,5870875,205900,6617075,283200


### Tabla 7: Ingresos familiares por aglomerado

In [21]:
info = df[['HOGAR_REF_ID', 'AGLOMERADO', 'P47T_hogar']].drop_duplicates().groupby(['AGLOMERADO'])['P47T_hogar'].describe()
info = info.reset_index().merge(aglo_labels).set_index('NOMAGLO').sort_index()

info['count'] = (info['count']/frac/4).fillna(0).astype(int)
info[['mean','std','min','25%','50%','75%','max']] = info[['mean','std','min','25%','50%','75%','max']]/ipc_10_16


In [29]:
show = info.copy()
show[['mean','std','min','25%','50%','75%','max']] = show[['mean','std','min','25%','50%','75%','max']].round(-1).astype(int)
show = show.drop('std', axis = 1)
show.to_csv('./../data/results/Tabla7.csv')
show

,AGLOMERADO,count,mean,min,25%,50%,75%,max
NOMAGLO,,,,,,,,
Bahia Blanca,3,100675,4790,0,2170,3910,6240,52470
CABA,32,1145700,6090,0,2640,4890,8110,148280
Catamarca,22,45300,4410,0,2150,3680,5760,39960
Cdro Rivadavia,9,49350,7700,0,3830,6650,10230,44670
Concordia,14,43050,4020,0,1840,3270,5410,23110
Cordoba,13,449250,4960,0,2230,4030,6560,74860
Corrientes,12,92525,3850,0,1610,2930,5170,32250
Formosa,15,58275,3720,0,1760,2940,4860,29970
Fuera de aglom.,0,4294875,3920,0,1640,3030,5180,105110


In [ ]:
# info
# # info.name = 'Cant.'
# # info = info.reset_index()
# # # info = info.rename(columns = {0: 'Cant.'})
# # ingresos_labels = pd.read_csv('./../data/info/ingresos_labels.csv')

# # info = info.merge(aglo_labels).merge()
# # info = info.set_index(['NOMAGLO', 'Ingresos'])['Cant.'].unstack()#[['count', 'mean', '25%', '50%', '75%']]
# # info = (info/frac/4).fillna(0).astype(int)

# # info.sort_index()

In [ ]:
# info = df.groupby(['AGLOMERADO'])['CBA'].describe().reset_index()
# info = info.merge(aglo_labels)#.merge(condact_labels).merge(cat_ocup_inac_labels, how = 'left')
# info = info.set_index('NOMAGLO')[['count', 'mean', '25%', '50%', '75%']]

# col_mon = ['mean', '25%', '50%', '75%']
# info[col_mon] = info[col_mon]/ipc_10_16
# info['count'] = info['count']/frac/4
# info = info.round(-1).astype(int)
# info.columns = ['Poblacion', 'CBA promedio', '25%', '50%', '75%']
# info.sort_index()